<a href="https://colab.research.google.com/github/ReevaKanakhara/Dwarka-Underwater-Satellite-Analysis/blob/main/DwarkaFinalCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install earthengine-api geemap folium

# Import required libraries
import ee
import folium
import geemap

# Authenticate Google Earth Engine
ee.Authenticate(auth_mode='notebook')

# Initialize Earth Engine
ee.Initialize(project='archaeology-research-paper')

# Define the Dwarka region (Approximate bounding box)
dwarka_region = ee.Geometry.Rectangle([68.9, 22.2, 69.1, 22.4])

# Load Sentinel-2 satellite imagery
sentinel = ee.ImageCollection('COPERNICUS/S2') \
    .filterBounds(dwarka_region) \
    .filterDate('2023-01-01', '2023-12-31') \
    .sort('CLOUDY_PIXEL_PERCENTAGE') \
    .first()

# Apply NDWI (Normalized Difference Water Index)
ndwi = sentinel.normalizedDifference(['B3', 'B8']).rename('NDWI')

# Mask out non-water areas (Keep only high NDWI values)
ndwi_threshold = ndwi.gt(0.3)  # Adjust threshold for ruins
masked_ndwi = ndwi.updateMask(ndwi_threshold)

# Create an interactive map
Map = geemap.Map(center=[22.3, 69.0], zoom=13)

# Add the Sentinel-2 image for reference
Map.addLayer(sentinel, {'bands': ['B11', 'B3', 'B2'], 'min': 0, 'max': 4000, 'gamma': 1.2}, 'Enhanced Sentinel-2')

# Add only the **underwater ruins** (masked NDWI)
Map.addLayer(masked_ndwi, {'palette': ['cyan'], 'min': 0, 'max': 1}, 'Underwater Ruins')

# Display the map
Map
